
# 🧬 Основы Python для биоинформатики
**База для ноутбука с графами, DFS, OLC и топологической сортировкой**

Этот ноутбук коротко покрывает всё, что нужно для выполнения заданий:
- переменные и типы,
- строки и срезы,
- списки, словари и множества,
- циклы и условия,
- функции,
- вложенные циклы для поиска перекрытий,
- минимальные заготовки: стековый DFS и топологическая сортировка (Kahn).

> Запускайте ячейки по порядку. Задания помечены как **Задание**. 



## 1. Переменные и типы данных
Основные типы:
- `int`, `float` — числа,
- `str` — строки,
- `list` — списки,
- `dict` — словари,
- `set` — множества,
- `bool` — логический тип (`True/False`).


In [ ]:

x = 5              # int
pi = 3.14          # float
seq = "ATCG"       # str
reads = ["ATCG", "TCGA"]  # list
overlaps = {"ATCG": ["TCGA"]}  # dict
unique_bases = {"A", "T", "C", "G"}  # set
flag = True        # bool

print(type(x), type(pi), type(seq), type(reads), type(overlaps), type(unique_bases), type(flag))



## 2. Строки: индексация и срезы
Строки важны для работы с ДНК:
- индексы с 0,
- срезы — `s[a:b]`, `s[:k]`, `s[-k:]`,
- операции: конкатенация, поиск подстроки.


In [ ]:

seq = "ATTACG"

print("Первый символ:", seq[0])
print("Последние три:", seq[-3:])
print("Без первых двух:", seq[2:])
print("Объединение:", seq + "TACG")
print("Длина:", len(seq))
print("'TAC' in seq?", "TAC" in seq)



## 3. Условия (`if / elif / else`) и логика
Пример: посчитаем GC-контент.


In [ ]:

seq = "ATCGGCGC"

gc = 0
for base in seq:
    if base in ["G", "C"]:
        gc += 1

gc_content = gc / len(seq)
print("GC-контент:", round(gc_content * 100, 2), "%")



## 4. Списки и циклы
Списки — упорядоченные коллекции. По ним удобно итерироваться.


In [ ]:

reads = ["ATTAC", "TACG", "CGT"]
print("Первый элемент:", reads[0])
reads.append("ACG")
print("После добавления:", reads)

print("Перебор:")
for r in reads:
    print("  read:", r)



## 5. Функции
Функции помогают переиспользовать код и структурировать решения.


In [ ]:

def gc_content(seq: str) -> float:
    """Возвращает долю G и C в последовательности."""
    gc = sum(1 for b in seq if b in "GC")
    return gc / len(seq)

print(gc_content("ATCG"))
print(gc_content("GGGGCCCC"))



## 6. Словари (`dict`) — представление графов
Словарь можно использовать как граф: ключ — вершина, значение — список соседей.


In [ ]:

graph = {"A": ["B", "C"], "B": ["D"], "C": [], "D": []}
print("Соседи A:", graph["A"])

# Добавим новую вершину
graph["E"] = ["A"]
print("Теперь есть вершина E:", graph)



## 7. Множества (`set`) — учёт посещённых вершин
Множество не хранит дубликаты и проверяет принадлежность за амортизированное O(1).


In [ ]:

visited = set()
visited.add("A")
visited.add("B")
visited.add("A")  # дубликат не добавится
print("visited =", visited)
print("'A' in visited?", "A" in visited)



## 8. Вложенные циклы — поиск перекрытий
Частая операция: сравнить каждую строку с каждой.


In [ ]:

reads = ["ATTAC", "TACG", "CGT"]
k = 2  # длина перекрытия
pairs = []
for a in reads:
    for b in reads:
        if a != b and a[-k:] == b[:k]:
            pairs.append((a, b))
pairs



### 🧪 Задание 1
Напишите функцию `count_kmer(seq, kmer)`, которая считает, сколько раз подстрока `kmer` встречается в `seq` (с перекрытиями).


In [ ]:

# TODO: реализуйте count_kmer(seq, kmer)
def count_kmer(seq: str, kmer: str) -> int:
    count = 0
    # ваш код здесь
    for i in range(len(seq) - len(kmer) + 1):
        if seq[i:i+len(kmer)] == kmer:
            count += 1
    return count

# проверка
assert count_kmer("ATATAT", "ATA") == 2
assert count_kmer("AAAAA", "AA") == 4
print("OK")



## 9. Итеративный DFS (без рекурсии)
Используем стек (обычный список) для обхода графа в глубину.


In [ ]:

def dfs_iter(graph: dict, start) -> list:
    visited = set()
    order = []
    stack = [start]
    while stack:
        node = stack.pop()
        if node not in visited:
            visited.add(node)
            order.append(node)
            # чтобы порядок был "естественным", добавим соседей в обратном порядке
            for neigh in reversed(graph.get(node, [])):
                if neigh not in visited:
                    stack.append(neigh)
    return order

graph = {"A": ["B", "C"], "B": ["D"], "C": [], "D": []}
print("DFS order:", dfs_iter(graph, "A"))



## 10. Топологическая сортировка (алгоритм Kahn)
Работает для ориентированного ациклического графа (DAG).



## 11. Вспомогательное: функция перекрытия строк
Возвращает длину перекрытия конца `a` и начала `b`.


In [ ]:

def overlap(a: str, b: str, min_overlap: int = 1) -> int:
    olen = 0
    for k in range(min_overlap, min(len(a), len(b)) + 1):
        if a.endswith(b[:k]):
            olen = k
    return olen

# проверка
assert overlap("ATTAC", "TACG", 2) == 3
assert overlap("TACG", "CGT", 1) == 2
print("OK")



### 🧪 Задание 2: Постройте граф перекрытий
Функция должна вернуть словарь `рид -> список ридов, которые могут идти следом` (по перекрытию ≥ `min_overlap`).



### 🧪 Задание 3: Сборка по топологическому порядку
1) Найдите топологический порядок ридов в графе перекрытий.  
2) Пройдите по порядку и собирайте строку, добавляя неперекрывающуюся часть.


In [ ]:

def assemble_from_order(order, reads_dict):
    if not order:
        return ""
    seq = reads_dict[order[0]]
    for i in range(1, len(order)):
        a = reads_dict[order[i-1]]
        b = reads_dict[order[i]]
        olen = overlap(a, b, min_overlap=1)
        seq += b[olen:]
    return seq

reads_dict = {r: r for r in reads}
order = topological_sort_kahn(ov_graph)
assembled = assemble_from_order(order, reads_dict)
print("Порядок:", order)
print("Сборка:", assembled)



## ✅ Итого
Вы освоили базовые конструкции Python, необходимые для работы с графами и строками в задачах мини-OLC:
- строки, списки, словари, множества;
- циклы, условия, функции;
- стековый DFS (идея) и топологическая сортировка (Kahn);
- поиск перекрытий и сборка строки по порядку.

Теперь можно переходить к основному ноутбуку с OLC/DFS/Topological Sort и решать задания! 
